# Import Library

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt

## Explanation of Dataset

In [ ]:
import os
print("Jumlah Data Train Tiap Kelas")
print('Jumlah gambar Normal :', len(os.listdir('../input/dataset-penyakit/Dataset_Final/train/Normal')))
print('Jumlah gambar Eksim  :', len(os.listdir('../input/dataset-penyakit/Dataset_Final/train/Eksim')))
print('Jumlah gambar Herpes :', len(os.listdir('../input/dataset-penyakit/Dataset_Final/train/Herpes')))
print('Jumlah gambar Jerawat:', len(os.listdir('../input/dataset-penyakit/Dataset_Final/train/Jerawat')))
print('Jumlah gambar Kudis  :', len(os.listdir('../input/dataset-penyakit/Dataset_Final/train/Kudis')))
print('Jumlah gambar Rosasea:', len(os.listdir('../input/dataset-penyakit/Dataset_Final/train/Rosasea')))

print("\nJumlah Data Test Tiap Kelas")
print('Jumlah gambar Normal :', len(os.listdir('../input/dataset-penyakit/Dataset_Final/test/Normal')))
print('Jumlah gambar Eksim  :', len(os.listdir('../input/dataset-penyakit/Dataset_Final/test/Eksim')))
print('Jumlah gambar Herpes :', len(os.listdir('../input/dataset-penyakit/Dataset_Final/test/Herpes')))
print('Jumlah gambar Jerawat:', len(os.listdir('../input/dataset-penyakit/Dataset_Final/test/Jerawat')))
print('Jumlah gambar Kudis  :', len(os.listdir('../input/dataset-penyakit/Dataset_Final/test/Kudis')))
print('Jumlah gambar Rosasea:', len(os.listdir('../input/dataset-penyakit/Dataset_Final/test/Rosasea')))

## Pre-processing Dataset

In [ ]:
IMAGE_SIZE = 150

train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "../input/dataset-penyakit/Dataset_Final/train",
    shuffle = True,
    image_size = (IMAGE_SIZE,IMAGE_SIZE),
    batch_size = 23,
    validation_split = 0.2,
    seed = 123,
    subset='training'
)

val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "../input/dataset-penyakit/Dataset_Final/train",
    shuffle = True,
    image_size = (IMAGE_SIZE,IMAGE_SIZE),
    batch_size = 23,
    validation_split = 0.2,
    seed = 123,
    subset='validation'
)

test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "../input/dataset-penyakit/Dataset_Final/test",
    image_size = (IMAGE_SIZE,IMAGE_SIZE),
    batch_size = 27,
)

In [ ]:
class_names = train_dataset.class_names
class_names

In [ ]:
plt.figure(figsize = (15,10))
for image_batch, label_batch in train_dataset.take(1):
   for i in range(12):
     ax = plt.subplot(3,4,i+1)
     plt.imshow(image_batch[i].numpy().astype("uint8"))
     plt.title(class_names[label_batch[i]], fontsize = 10)
     plt.axis("off")

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.cache().prefetch(buffer_size=AUTOTUNE)
validation_dataset = val_dataset.cache().prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
data_augmentation = tf.keras.Sequential([
                     layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
                     layers.experimental.preprocessing.RandomRotation(0.2),
])

In [ ]:
for image, _ in train_dataset.take(1):
  plt.figure(figsize=(10, 10))
  first_image = image[0]
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    augmented_image = data_augmentation(tf.expand_dims(first_image, 0))
    plt.imshow(augmented_image[0] / 255)
    plt.axis('off')

# Build a Pre-Trained Model

In [ ]:
preprocess_input = tf.keras.applications.resnet.preprocess_input

base_model = tf.keras.applications.resnet50.ResNet50(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(150, 150, 3),
    include_top=False,
)  # Do not include the ImageNet classifier at the top.

# Freeze the base_model
base_model.trainable = False

# Create new model on top
inputs = tf.keras.Input(shape=(150, 150, 3))
x = data_augmentation(inputs)  # Apply random data augmentation
x = preprocess_input(x)

# The base model contains batchnorm layers. We want to keep them in inference mode
# when we unfreeze the base model for fine-tuning, so we make sure that the
# base_model is running in inference mode here.
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)  # Regularize with dropout
x = tf.keras.layers.Dense(6, name='outputLayer')(x)
outputs = tf.keras.layers.Activation(activation="softmax", dtype=tf.float32, name='act_layer')(x)
model = tf.keras.Model(inputs, outputs, name = "FeatureExtractionModel")

model.summary()

In [ ]:
base_model.summary()

# Train Top Layer of Pre-Trained Model

In [ ]:
model.compile(
    optimizer= tf.keras.optimizers.Adam(),
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
EPOCHS = 100
history = model.fit(
    train_dataset,
    epochs = EPOCHS,
    batch_size = 23,
    validation_data = validation_dataset,
    verbose = 1,
) 

In [ ]:
scores = model.evaluate(test_dataset)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(range(EPOCHS), acc, label='Training Accuracy')
plt.plot(range(EPOCHS), val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(range(EPOCHS), loss, label='Training Loss')
plt.plot(range(EPOCHS), val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# Fine Tuning

In [ ]:
base_model.trainable = True

In [ ]:
for layer in model.layers[4].layers:
    if isinstance(layer, layers.BatchNormalization):
        layer.trainable = False

In [ ]:
model.summary()

In [ ]:
model.compile(
    optimizer= tf.keras.optimizers.Adam(1e-5),
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
FINE_TUNING_EPOCHS = 100
TOTAL_EPOCHS = EPOCHS + FINE_TUNING_EPOCHS

history.fine = model.fit(
    train_dataset,
    epochs=TOTAL_EPOCHS,
    initial_epoch=history.epoch[-1],
    batch_size = 23,
    validation_data = validation_dataset,
    verbose = 1,
) 

In [ ]:
scores = model.evaluate(test_dataset)

In [ ]:
acc = history.fine.history['accuracy']
val_acc = history.fine.history['val_accuracy']

loss = history.fine.history['loss']
val_loss = history.fine.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(range(101), acc, label='Training Accuracy')
plt.plot(range(101), val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(range(101), loss, label='Training Loss')
plt.plot(range(101), val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# Predict

In [ ]:
for images_batch, labels_batch in test_dataset.take(1):
    
    first_image = images_batch[6].numpy().astype('uint8')
    first_label = labels_batch[6].numpy()
    
    print("first image to predict")
    plt.imshow(first_image)
    print("actual label:",class_names[first_label])
    
    batch_prediction = model.predict(images_batch)
    print("predicted label:",class_names[np.argmax(batch_prediction[6])])

In [ ]:
def predict(model, img):
    img_array = tf.keras.preprocessing.image.img_to_array(images[i].numpy())
    img_array = tf.expand_dims(img_array, 0)

    predictions = model.predict(img_array)

    predicted_class = class_names[np.argmax(predictions[0])]
    confidence = round(100 * (np.max(predictions[0])), 2)
    return predicted_class, confidence

In [ ]:
plt.figure(figsize=(15, 15))
for images, labels in test_dataset.take(3):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        
        predicted_class, confidence = predict(model, images[i].numpy())
        actual_class = class_names[labels[i]] 
        
        plt.title(f"Actual: {actual_class},\n Predicted: {predicted_class}.\n Confidence: {confidence}%")
        
        plt.axis("off")

In [ ]:
model.save("Model_Resnet50_85%.h5")

In [ ]:
my_reloaded_model = tf.keras.models.load_model(('./Model_Resnet50_85%.h5'))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from keras.preprocessing import image

img = Image.open('../input/dataset-penyakit/Dataset_Final/test/Rosasea/rosasea006.jpeg')
resize_image = img.resize((150, 150))
plt.imshow(resize_image)

x = image.img_to_array(resize_image)
x = np.expand_dims(x, axis = 0)
images = np.vstack([x])

prediction = my_reloaded_model.predict(images)
print('\nThis image belongs to class', class_names[np.argmax(prediction[0])])